In [46]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q

In [47]:
client = Elasticsearch('https://67a7441549120daa2dbeef8ac4f5bb2e.us-east-1.aws.found.io:9243')

In [48]:
client.ping()

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


True

In [4]:
# upload the data to elasticsearch
try:
  client.indices.delete('plotly_datasets')
except:
  pass

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [5]:
client.indices.create('plotly_datasets')

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'acknowledged': True}

In [6]:
client.indices.put_mapping(
  index='plotly_datasets',
  doc_type='consumer_complaints',    
  body={
    'properties': {
       'Company': {'type': 'string'},
       'Company response': {'type': 'string'},
       'Complaint ID': {'type': 'integer'},
       'Consumer disputed?': {'type': 'string'},
       'Date received': {'type': 'date', 'format': 'strict_date_optional_time'},
       'Date sent to company': {'type': 'date', 'format': 'strict_date_optional_time'},
       'Issue': {'type': 'string'},
       'Product': {'type': 'string'},
       'State': {'type': 'string'},
       'Sub-issue': {'type': 'string'},
       'Sub-product': {'type': 'string'},
       'Timely response?': {'type': 'string'},
       'ZIP code': {'type': 'integer'}
    }
  }
)

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'acknowledged': True}

In [139]:
client.indices.get_mapping()

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'plotly_datasets': {u'mappings': {u'consumer_complaints': {u'properties': {u'Company': {u'type': u'string'},
     u'Company response': {u'type': u'string'},
     u'Complaint ID': {u'type': u'integer'},
     u'Consumer disputed?': {u'type': u'string'},
     u'Date received': {u'format': u'strict_date_optional_time',
      u'type': u'date'},
     u'Date sent to company': {u'format': u'strict_date_optional_time',
      u'type': u'date'},
     u'Issue': {u'type': u'string'},
     u'Product': {u'type': u'string'},
     u'State': {u'type': u'string'},
     u'Sub-issue': {u'type': u'string'},
     u'Sub-product': {u'type': u'string'},
     u'Timely response?': {u'type': u'string'},
     u'ZIP code': {u'type': u'integer'}}},
   u'ebola_2014': {u'properties': {u'Country': {u'type': u'string'},
     u'Lat': {u'type': u'float'},
     u'Lon': {u'type': u'float'},
     u'Month': {u'type': u'integer'},
     u'Value': {u'type': u'float'},
     u'Year': {u'type': u'integer'},
     u'index': {u'type'

In [8]:
import pandas as pd
df = pd.DataFrame(pd.read_csv("/Users/Alexandre/Workstation/databases/26k-consumer-complaints.csv"))

//anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [9]:
df.head()

,Unnamed: 0,Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
0,0,1291006,Debt collection,NaN,Communication tactics,Frequent or repeated calls,TX,76119.0,2015-03-19,2015-03-19,"Premium Asset Services, LLC",In progress,Yes,NaN
1,1,1290580,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,TX,77479.0,2015-03-19,2015-03-19,Accounts Receivable Consultants Inc.,Closed with explanation,Yes,NaN
2,2,1290564,Mortgage,FHA mortgage,"Application, originator, mortgage broker",NaN,MA,2127.0,2015-03-19,2015-03-19,RBS Citizens,Closed with explanation,Yes,Yes
3,3,1291615,Credit card,NaN,Other,NaN,CA,92592.0,2015-03-19,2015-03-19,Navy FCU,In progress,Yes,NaN
4,4,1292165,Debt collection,Non-federal student loan,Cont'd attempts collect debt not owed,Debt resulted from identity theft,NaN,43068.0,2015-03-19,2015-03-19,Transworld Systems Inc.,In progress,Yes,NaN


In [10]:
df = df.drop('Unnamed: 0', axis=1)
columnsNaNtoNone = ['Company', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'State', 'Company', 'Company response', 'Timely response?', 'Consumer disputed?']  
columnsNaNtoZero = ['Complaint ID', 'ZIP code']
columnsNaNtoDate = ['Date received', 'Date sent to company']

In [11]:
df1 = df[columnsNaNtoNone].fillna(value='None')
df1.head()

,Company,Product,Sub-product,Issue,Sub-issue,State,Company,Company response,Timely response?,Consumer disputed?
0,"Premium Asset Services, LLC",Debt collection,None,Communication tactics,Frequent or repeated calls,TX,"Premium Asset Services, LLC",In progress,Yes,None
1,Accounts Receivable Consultants Inc.,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,TX,Accounts Receivable Consultants Inc.,Closed with explanation,Yes,None
2,RBS Citizens,Mortgage,FHA mortgage,"Application, originator, mortgage broker",None,MA,RBS Citizens,Closed with explanation,Yes,Yes
3,Navy FCU,Credit card,None,Other,None,CA,Navy FCU,In progress,Yes,None
4,Transworld Systems Inc.,Debt collection,Non-federal student loan,Cont'd attempts collect debt not owed,Debt resulted from identity theft,None,Transworld Systems Inc.,In progress,Yes,None


In [12]:
df2 = df[columnsNaNtoZero].fillna(value=0)
df2.head()

,Complaint ID,ZIP code
0,1291006,76119.0
1,1290580,77479.0
2,1290564,2127.0
3,1291615,92592.0
4,1292165,43068.0


In [13]:
df3 = df[columnsNaNtoDate].fillna(value='2000-01-01')
df3.head()

,Date received,Date sent to company
0,2015-03-19,2015-03-19
1,2015-03-19,2015-03-19
2,2015-03-19,2015-03-19
3,2015-03-19,2015-03-19
4,2015-03-19,2015-03-19


In [14]:
newdf = pd.concat([df1, df2, df3], axis=1, join='inner')
newdf.head()

,Company,Product,Sub-product,Issue,Sub-issue,State,Company,Company response,Timely response?,Consumer disputed?,Complaint ID,ZIP code,Date received,Date sent to company
0,"Premium Asset Services, LLC",Debt collection,None,Communication tactics,Frequent or repeated calls,TX,"Premium Asset Services, LLC",In progress,Yes,None,1291006,76119.0,2015-03-19,2015-03-19
1,Accounts Receivable Consultants Inc.,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,TX,Accounts Receivable Consultants Inc.,Closed with explanation,Yes,None,1290580,77479.0,2015-03-19,2015-03-19
2,RBS Citizens,Mortgage,FHA mortgage,"Application, originator, mortgage broker",None,MA,RBS Citizens,Closed with explanation,Yes,Yes,1290564,2127.0,2015-03-19,2015-03-19
3,Navy FCU,Credit card,None,Other,None,CA,Navy FCU,In progress,Yes,None,1291615,92592.0,2015-03-19,2015-03-19
4,Transworld Systems Inc.,Debt collection,Non-federal student loan,Cont'd attempts collect debt not owed,Debt resulted from identity theft,None,Transworld Systems Inc.,In progress,Yes,None,1292165,43068.0,2015-03-19,2015-03-19


In [15]:
newdf.tail()

,Company,Product,Sub-product,Issue,Sub-issue,State,Company,Company response,Timely response?,Consumer disputed?,Complaint ID,ZIP code,Date received,Date sent to company
28151,"Enhanced Recovery Company, LLC",Debt collection,None,Cont'd attempts collect debt not owed,Debt is not mine,TX,"Enhanced Recovery Company, LLC",Closed with non-monetary relief,Yes,No,1178184,75201.0,2015-01-01,2015-01-02
28152,DriveTime,Consumer loan,Vehicle loan,Taking out the loan or lease,None,GA,DriveTime,Closed with explanation,Yes,None,1178183,30043.0,2015-01-01,2015-01-07
28153,Regions,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",None,MN,Regions,Closed with non-monetary relief,Yes,No,1178182,55105.0,2015-01-01,2015-01-02
28154,Moneytree Inc,Payday loan,None,Can't stop charges to bank account,None,CA,Moneytree Inc,Closed with explanation,Yes,None,1178181,92509.0,2015-01-01,2015-01-07
28155,PayPal,Money transfers,Domestic (US) money transfer,Other transaction issues,None,OH,PayPal,Closed with explanation,Yes,Yes,1178180,45068.0,2015-01-01,2015-01-02


In [16]:
newdf.isnull().any()

Company                 False
Product                 False
Sub-product             False
Issue                   False
Sub-issue               False
State                   False
Company                 False
Company response        False
Timely response?        False
Consumer disputed?      False
Complaint ID            False
ZIP code                False
Date received           False
Date sent to company    False
dtype: bool

In [17]:
newdf.isnull().any().any() 

False

In [18]:
help(client.indices.get_mapping)

Help on method get_mapping in module elasticsearch.client.indices:

get_mapping(*args, **kwargs) method of elasticsearch.client.indices.IndicesClient instance
    Retrieve mapping definition of index or index/type.
    `<http://www.elastic.co/guide/en/elasticsearch/reference/current/indices-get-mapping.html>`_
    
    :arg index: A comma-separated list of index names
    :arg doc_type: A comma-separated list of document types
    :arg allow_no_indices: Whether to ignore if a wildcard indices
        expression resolves into no concrete indices. (This includes `_all`
        string or when no indices have been specified)
    :arg expand_wildcards: Whether to expand wildcard expression to concrete
        indices that are open, closed or both., default 'open', valid
        choices are: 'open', 'closed', 'none', 'all'
    :arg ignore_unavailable: Whether specified concrete indices should be
        ignored when unavailable (missing or closed)
    :arg local: Return local information, do

In [19]:
mappings = dict()
mappings

{}

In [20]:
for name in df.columns:
  mappings[name] = {'type': 'string'}
  
mappings

{'Company': {'type': 'string'},
 'Company response': {'type': 'string'},
 'Complaint ID': {'type': 'string'},
 'Consumer disputed?': {'type': 'string'},
 'Date received': {'type': 'string'},
 'Date sent to company': {'type': 'string'},
 'Issue': {'type': 'string'},
 'Product': {'type': 'string'},
 'State': {'type': 'string'},
 'Sub-issue': {'type': 'string'},
 'Sub-product': {'type': 'string'},
 'Timely response?': {'type': 'string'},
 'ZIP code': {'type': 'string'}}

In [21]:
# upload the data to elasticsearch
client.indices.put_mapping(
  index='plotly_datasets',
  doc_type='consumer_complaints',    
  body={
    'properties': {
      'Company': {'type': 'string'},
      'Company response': {'type': 'string'},
      'Complaint ID': {'type': 'integer'},
      'Consumer disputed?': {'type': 'string'},
      'Date received': {'type': 'date', 'format': 'strict_date_optional_time'},
      'Date sent to company': {'type': 'date', 'format': 'strict_date_optional_time'},
      'Issue': {'type': 'string'},
      'Product': {'type': 'string'},
      'State': {'type': 'string'},
      'Sub-issue': {'type': 'string'},
      'Sub-product': {'type': 'string'},
      'Timely response?': {'type': 'string'},
      'ZIP code': {'type': 'integer'}
    }
  }
)

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'acknowledged': True}

In [22]:
client.indices.get_mapping()

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'plotly_datasets': {u'mappings': {u'consumer_complaints': {u'properties': {u'Company': {u'type': u'string'},
     u'Company response': {u'type': u'string'},
     u'Complaint ID': {u'type': u'integer'},
     u'Consumer disputed?': {u'type': u'string'},
     u'Date received': {u'format': u'strict_date_optional_time',
      u'type': u'date'},
     u'Date sent to company': {u'format': u'strict_date_optional_time',
      u'type': u'date'},
     u'Issue': {u'type': u'string'},
     u'Product': {u'type': u'string'},
     u'State': {u'type': u'string'},
     u'Sub-issue': {u'type': u'string'},
     u'Sub-product': {u'type': u'string'},
     u'Timely response?': {u'type': u'string'},
     u'ZIP code': {u'type': u'integer'}}}}},
 u'sample-data': {u'mappings': {u'test-type': {u'properties': {u'aggs': {u'properties': {u'agg1': {u'properties': {u'aggs': {u'properties': {u'agg2': {u'properties': {u'sum': {u'properties': {u'field': {u'type': u'string'}}}}}}},
         u'histogram': {u'properties': 

In [23]:
bulk = []
for i in range(len(newdf)):
    bulk.append({'index': {'_index': 'plotly_datasets', '_type': 'consumer_complaints', '_id': str(i)}})
    bulk.append(newdf.iloc[i].to_dict())

In [24]:
response = client.bulk(bulk, request_timeout=100000)

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [25]:
response.keys()

[u'items', u'errors', u'took']

In [26]:
response['errors']

False

In [27]:
response['took']

3207

In [28]:
len(response['items'])

28156

In [142]:
response['items'][0]['index']['status'] == 201

True

In [143]:
errors = [res for res in response['items'] if res['index']['status'] != 201]
len(errors)

0

In [109]:
client.search(
  index='plotly_datasets',
  doc_type='consumer_complaints',
  body={
    'query': {
      'match_all': {}
    },
    'size':1
  }
)

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'_shards': {u'failed': 0, u'successful': 1, u'total': 1},
 u'hits': {u'hits': [{u'_id': u'0',
    u'_index': u'plotly_datasets',
    u'_score': 1.0,
    u'_source': {u'Company': u'Premium Asset Services, LLC',
     u'Company response': u'In progress',
     u'Complaint ID': 1291006,
     u'Consumer disputed?': u'None',
     u'Date received': u'2015-03-19',
     u'Date sent to company': u'2015-03-19',
     u'Issue': u'Communication tactics',
     u'Product': u'Debt collection',
     u'State': u'TX',
     u'Sub-issue': u'Frequent or repeated calls',
     u'Sub-product': u'None',
     u'Timely response?': u'Yes',
     u'ZIP code': 76119.0},
    u'_type': u'consumer_complaints'}],
  u'max_score': 1.0,
  u'total': 28156},
 u'timed_out': False,
 u'took': 1}

In [88]:
import requests
import json

In [51]:
domain = 'https://67a7441549120daa2dbeef8ac4f5bb2e.us-east-1.aws.found.io:9243'

In [95]:
body = json.dumps({
  "query": {
        "match_all" : { }
  },
  "size": 20
})


In [96]:
r = requests.post(domain + '/plotly_datasets/consumer_complaints/_search?format=json', data = body )
response = r.json()
len(response['hits']['hits'])

In [103]:
body = json.dumps({
  "query": {
        "match_all" : { }
  },
  "size": 10001
})

In [105]:
r = requests.post(domain + '/plotly_datasets/consumer_complaints/_search?format=json&scroll=1m', data = body )
response = r.json()
len(response['hits']['hits'])

10001

In [107]:
response.keys()

[u'hits', u'_shards', u'took', u'_scroll_id', u'timed_out']

In [117]:
response = client.search(
  index='sample-data',
  doc_type='test-type',
  body={
    'query': {
      'match_all': {}
    }
  }
)

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [145]:
client.ping()

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


True

In [160]:
# upload the data to elasticsearch
try:
  client.indices.delete('sample-data')
except:
  pass

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [161]:
client.indices.create('sample-data')

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'acknowledged': True}

In [162]:
client.indices.put_mapping(
  index='sample-data',
  doc_type='test-type',    
  body={
   'properties': {
     'my-boolean-1': {
       'type': 'boolean'
     },
     'my-boolean-2': {
       'type': 'boolean'
     },
     'my-date-1': {
       'format': 'strict_date_optional_time||epoch_millis',
       'type': 'date'
     },
     'my-date-2': {
       'format': 'strict_date_optional_time||epoch_millis',
       'type': 'date'
     },
     'my-geo-point-1': {
       'type': 'geo_point'
     },
     'my-geo-point-2': {
       'type': 'geo_point'
     },
     'my-number-1': {
       'type': 'long'
     },
     'my-number-2': {
       'type': 'long'
     },
     'my-string-1': {
       'type': 'string'
     },
     'my-string-2': {
       'type': 'string'
     }
   }
  }
)             

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'acknowledged': True}

In [163]:
sample_data = pd.DataFrame(
    columns=[
        'my-string-1', 'my-string-2',
        'my-number-1', 'my-number-2',
        'my-date-1', 'my-date-2',
        'my-boolean-1', 'my-boolean-2',
        'my-geo-point-1', 'my-geo-point-2'
        # TODO - do these        
        # 'my-geo-point-2',
#         'my-geo-shape-1', 'my-geo-shape-2', 
#         'my-ipv4-1', 'my-ipv4-2',
        # skipping array, object, nested, completion, token, mapper-murmur3
    ],
    data=zip(*[
        ['NYC', 'NYC', 'NYC', 'Paris', 'Paris', 'Tokyo', 'Tokyo', 'Tokyo', 'SF', 'Sf', 'Montreal'],
        ['USA', 'USA', 'USA', 'France', 'France', 'Japan', 'Japan', 'Japan', 'USA', 'USA', 'Canada'],
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
        [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
    
        [
            '2015-01-01T12:30:40Z',
            '2015-10-04T12:35:10Z',
            '2015-02-02T12:45:02Z',

            '2015-04-12T12:55:05Z',
            '2016-02-15T08:10:10Z',

            '2016-01-10T06:05:02Z',
            '2016-02-11T07:02:01Z',
            '2012-03-13T05:01:05Z',            
            
            '2010-01-16T01:10:50Z',            
            '2011-04-19T02:15:38Z',            

            '2012-02-20T03:01:28Z'
        ],
        [
            '1915-01-01T12:30:40Z',
            '1915-10-04T12:35:10Z',
            '1915-02-02T12:45:02Z',

            '1915-04-12T12:55:05Z',
            '1916-02-15T08:10:10Z',

            '1916-01-10T06:05:02Z',
            '1916-02-11T07:02:01Z',
            '1912-03-13T05:01:05Z',            
            
            '1910-01-16T01:10:50Z',            
            '1911-04-19T02:15:38Z',            

            '1912-02-20T03:01:28Z'
        ],
        [True, True, True, False, False, True, True, True, False, False, True],
        [True, False, True, False, True, False, True, False, True, False, True],
        
        [
            [10, 10],
            [11, 11],           
            [12, 12],
            
            [20, 20],
            [21, 21],           

            [30, 30],
            [31, 31],        
            [32, 32],
            
            [40, 40],
            [41, 41],        

            [50, 50]                    
        ],
        
        [
            [-10, -10],
            [-11, -11],           
            [-12, -12],
            
            [-20, -20],
            [-21, -21],           

            [-30, -30],
            [-31, -31],        
            [-32, -32],
            
            [-40, -40],
            [-41, -41],        

            [-50, -50]                    
        ]        
    ])
)

In [164]:
sample_data

,my-string-1,my-string-2,my-number-1,my-number-2,my-date-1,my-date-2,my-boolean-1,my-boolean-2,my-geo-point-1,my-geo-point-2
0,NYC,USA,1,10,2015-01-01T12:30:40Z,1915-01-01T12:30:40Z,True,True,"[10, 10]","[-10, -10]"
1,NYC,USA,2,20,2015-10-04T12:35:10Z,1915-10-04T12:35:10Z,True,False,"[11, 11]","[-11, -11]"
2,NYC,USA,3,30,2015-02-02T12:45:02Z,1915-02-02T12:45:02Z,True,True,"[12, 12]","[-12, -12]"
3,Paris,France,4,40,2015-04-12T12:55:05Z,1915-04-12T12:55:05Z,False,False,"[20, 20]","[-20, -20]"
4,Paris,France,5,50,2016-02-15T08:10:10Z,1916-02-15T08:10:10Z,False,True,"[21, 21]","[-21, -21]"
5,Tokyo,Japan,6,60,2016-01-10T06:05:02Z,1916-01-10T06:05:02Z,True,False,"[30, 30]","[-30, -30]"
6,Tokyo,Japan,7,70,2016-02-11T07:02:01Z,1916-02-11T07:02:01Z,True,True,"[31, 31]","[-31, -31]"
7,Tokyo,Japan,8,80,2012-03-13T05:01:05Z,1912-03-13T05:01:05Z,True,False,"[32, 32]","[-32, -32]"
8,SF,USA,9,90,2010-01-16T01:10:50Z,1910-01-16T01:10:50Z,False,True,"[40, 40]","[-40, -40]"
9,Sf,USA,10,100,2011-04-19T02:15:38Z,1911-04-19T02:15:38Z,False,False,"[41, 41]","[-41, -41]"


In [165]:
bulk = []
for i in range(len(sample_data)):
    bulk.append({'index': {'_index': 'sample-data', '_type': 'test-type', '_id': str(i)}})

    row = sample_data.iloc[i].to_dict()
    row['my-boolean-1'] = bool(sample_data.iloc[i].to_dict()['my-boolean-1'] )
    row['my-boolean-2'] = bool(sample_data.iloc[i].to_dict()['my-boolean-2'] )
    bulk.append(row)

In [166]:
response = client.bulk(bulk, request_timeout=100000)

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [167]:
response.keys()

[u'items', u'errors', u'took']

In [168]:
response['errors']

False

In [157]:
response['items']

[{u'index': {u'_id': u'0',
   u'_index': u'sample-data',
   u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
   u'_type': u'test-type',
   u'_version': 1,
   u'status': 201}},
 {u'index': {u'_id': u'1',
   u'_index': u'sample-data',
   u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
   u'_type': u'test-type',
   u'_version': 1,
   u'status': 201}},
 {u'index': {u'_id': u'2',
   u'_index': u'sample-data',
   u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
   u'_type': u'test-type',
   u'_version': 1,
   u'status': 201}},
 {u'index': {u'_id': u'3',
   u'_index': u'sample-data',
   u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
   u'_type': u'test-type',
   u'_version': 1,
   u'status': 201}},
 {u'index': {u'_id': u'4',
   u'_index': u'sample-data',
   u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
   u'_type': u'test-type',
   u'_version': 1,
   u'status': 201}},
 {u'index': {u'_id': u'5',
   u'_index': u'sample-data',
   u'_shards'